# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** My final architecture was a Resnet50 as CNN encoder and an single layer LSTM as RNN decoder. I also tried increasing the `num_layers` of the LSTM to $2$ and $3$ with `dropout` as $0.01$, $0.02$ and $0.03$. But,in every case the predicted captions were same for all the images. This was the reason why I finally went with the single layer LSTM architecture. 

I chose the `embed_size` and `hidden_size` value both of them as 512 as used in the paper [this paper](https://arxiv.org/pdf/1411.4555.pdf). I trained the model for 2 epochs. For the `epoch=1`, I used a `batch_size` as `64` and then increased to `128` for `epoch=2`. The initial `batch_size` of 64 was inspired from [this paper](https://arxiv.org/pdf/1502.03044.pdf). The training stats can be viewed for $1$st `epoch` in **training_log.txt** and $2$nd `epoch` in **training_log_4.txt**.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I went with the default values, since the `resnet50` architecture is trained on images of shape (224, 224, 3). I believe, if I had used a different image size, then the predictions of the encoder would have been affected. Also, I was getting pretty decent captions with the default values in the first model itself, so I did not switch. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** Since, we are using a pretrained model for the encoder, it doesn't make sense to train them again. Although, we would like to fine tune the linear layer (`encoder.embed`) that we add to the `resnet50` architecture.
For the decoder, since we are not using a pretrained network, we would have to train the complete decoder network(`decoder`).

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I used `Adam` optimiser for all the model that I experimented with. The reason for using Adam is that **(1)** It can lead to convergence faster than SGD esecially in deep networks. **(2)** It uses adaptive learning rates which has found to be more efficient in updating the weights. 

I trained the final model for $1$st `epoch` with `learning_rate=0.001`. Then, I retrained the model for another `epoch` with `learning_rate=0.002`.  

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128         # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log_4.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)



Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.02s)
creating index...


  0%|          | 1147/414113 [00:00<01:16, 5407.96it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:05<00:00, 6300.66it/s]


In [2]:
# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
print (vocab_size)
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# loading previous models for retraining
# encoder.load_state_dict(torch.load('./models/encoder-1.pkl'))
# decoder.load_state_dict(torch.load('./models/decoder-1.pkl'))


# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.002)
# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)
print ('total no of steps: {0}'.format(total_step))

6335


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:05<00:00, 20020235.38it/s]


total no of steps: 3236


In [3]:
encoder

EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64, 256, ker

In [4]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.002
    weight_decay: 0
)

In [5]:
decoder

DecoderRNN(
  (embed): Embedding(6335, 512)
  (lstm): LSTM(512, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=6335, bias=True)
)

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [6]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        print ('CNN features: {0} and captions: {1}'.format(features.shape, captions.shape))
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
#             to save intermediate models after 100 steps, so as to obtain quicker feedbacks
#             torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-step-v3-%d.pkl' % i_step))
#             torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-step-v3-%d.pkl' % i_step))
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-v1-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-v1-%d.pkl' % epoch))

# Close the training log file.
f.close()

CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [1/3236], Loss: 1.9984, Perplexity: 7.3770CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [2/3236], Loss: 2.1711, Perplexity: 8.7683CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 10])
Epoch [1/1], Step [3/3236], Loss: 2.3664, Perplexity: 10.6592CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [4/3236], Loss: 2.0900, Perplexity: 8.0848CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [5/3236], Loss: 2.2010, Perplexity: 9.0339CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 17])
Epoch [1/1], Step [6/3236], Loss: 2.4364, Perplexity: 11.4321CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [7/3236], Loss: 2.1467, Perplexity: 8.5562CNN features: torch.Size([128, 512]) and captions: torch.Size([128,

Epoch [1/1], Step [62/3236], Loss: 2.3490, Perplexity: 10.4754CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [63/3236], Loss: 2.2699, Perplexity: 9.6787CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [64/3236], Loss: 2.1559, Perplexity: 8.6352CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [65/3236], Loss: 2.3503, Perplexity: 10.4890CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 15])
Epoch [1/1], Step [66/3236], Loss: 2.2798, Perplexity: 9.7752CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 15])
Epoch [1/1], Step [67/3236], Loss: 2.3153, Perplexity: 10.1282CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [68/3236], Loss: 2.3015, Perplexity: 9.9891CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [69/3236], Loss: 2.0769, Perplexity: 7.97

Epoch [1/1], Step [123/3236], Loss: 2.1169, Perplexity: 8.3051CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [124/3236], Loss: 2.1038, Perplexity: 8.1973CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 21])
Epoch [1/1], Step [125/3236], Loss: 2.9515, Perplexity: 19.1355CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [126/3236], Loss: 2.2203, Perplexity: 9.2098CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 10])
Epoch [1/1], Step [127/3236], Loss: 2.3491, Perplexity: 10.4760CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [128/3236], Loss: 2.2218, Perplexity: 9.2237CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [129/3236], Loss: 2.0777, Perplexity: 7.9857CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [130/3236], Loss: 2.1739, Perplexit

Epoch [1/1], Step [184/3236], Loss: 2.1573, Perplexity: 8.6474CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 18])
Epoch [1/1], Step [185/3236], Loss: 2.5335, Perplexity: 12.5972CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [186/3236], Loss: 2.1890, Perplexity: 8.9258CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [187/3236], Loss: 2.0820, Perplexity: 8.0204CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 21])
Epoch [1/1], Step [188/3236], Loss: 2.9390, Perplexity: 18.8978CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [189/3236], Loss: 2.2108, Perplexity: 9.1231CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [190/3236], Loss: 2.0433, Perplexity: 7.7158CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [191/3236], Loss: 2.1975, Perplexit

Epoch [1/1], Step [245/3236], Loss: 2.5295, Perplexity: 12.5473CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 16])
Epoch [1/1], Step [246/3236], Loss: 2.5010, Perplexity: 12.1948CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [247/3236], Loss: 2.0748, Perplexity: 7.9630CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [248/3236], Loss: 2.0717, Perplexity: 7.9384CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [249/3236], Loss: 2.1449, Perplexity: 8.5414CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [250/3236], Loss: 2.1647, Perplexity: 8.7117CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [251/3236], Loss: 2.2679, Perplexity: 9.6594CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 18])
Epoch [1/1], Step [252/3236], Loss: 2.7301, Perplexit

...

Epoch [1/1], Step [2657/3236], Loss: 2.0632, Perplexity: 7.8712CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [2658/3236], Loss: 1.9929, Perplexity: 7.3369CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 15])
Epoch [1/1], Step [2659/3236], Loss: 2.2098, Perplexity: 9.1143CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [2660/3236], Loss: 2.0342, Perplexity: 7.6462CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2661/3236], Loss: 1.9049, Perplexity: 6.7185CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2662/3236], Loss: 1.9569, Perplexity: 7.0776CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 16])
Epoch [1/1], Step [2663/3236], Loss: 2.2407, Perplexity: 9.3999CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2664/3236], Loss: 2.0894, Per

Epoch [1/1], Step [2777/3236], Loss: 2.0072, Perplexity: 7.4424CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 10])
Epoch [1/1], Step [2778/3236], Loss: 2.2982, Perplexity: 9.9565CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2779/3236], Loss: 2.0845, Perplexity: 8.0407CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2780/3236], Loss: 2.0578, Perplexity: 7.8286CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [2781/3236], Loss: 2.0457, Perplexity: 7.7346CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [2782/3236], Loss: 2.0099, Perplexity: 7.4625CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [2783/3236], Loss: 1.9782, Perplexity: 7.2296CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 17])
Epoch [1/1], Step [2784/3236], Loss: 2.2955, Per

Epoch [1/1], Step [2897/3236], Loss: 2.0824, Perplexity: 8.0236CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [2898/3236], Loss: 2.0417, Perplexity: 7.7036CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 16])
Epoch [1/1], Step [2899/3236], Loss: 2.3057, Perplexity: 10.0310CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [2900/3236], Loss: 2.0398, Perplexity: 7.6890
CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [2901/3236], Loss: 1.8864, Perplexity: 6.5955CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [2902/3236], Loss: 1.9283, Perplexity: 6.8779CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [2903/3236], Loss: 2.1014, Perplexity: 8.1775CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [2904/3236], Loss: 2.0557, P

Epoch [1/1], Step [3017/3236], Loss: 2.1241, Perplexity: 8.3650CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 24])
Epoch [1/1], Step [3018/3236], Loss: 3.0580, Perplexity: 21.2846CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 13])
Epoch [1/1], Step [3019/3236], Loss: 1.9880, Perplexity: 7.3012CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [3020/3236], Loss: 2.1564, Perplexity: 8.6403CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 10])
Epoch [1/1], Step [3021/3236], Loss: 2.3523, Perplexity: 10.5096CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 17])
Epoch [1/1], Step [3022/3236], Loss: 2.4586, Perplexity: 11.6883CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [3023/3236], Loss: 2.0090, Perplexity: 7.4556CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [3024/3236], Loss: 2.2102, 

Epoch [1/1], Step [3137/3236], Loss: 2.0760, Perplexity: 7.9723CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 11])
Epoch [1/1], Step [3138/3236], Loss: 2.2402, Perplexity: 9.3948CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 14])
Epoch [1/1], Step [3139/3236], Loss: 2.0569, Perplexity: 7.8220CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [3140/3236], Loss: 1.9376, Perplexity: 6.9421CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 12])
Epoch [1/1], Step [3141/3236], Loss: 1.9052, Perplexity: 6.7209CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 15])
Epoch [1/1], Step [3142/3236], Loss: 2.2383, Perplexity: 9.3778CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 15])
Epoch [1/1], Step [3143/3236], Loss: 2.1693, Perplexity: 8.7521CNN features: torch.Size([128, 512]) and captions: torch.Size([128, 17])
Epoch [1/1], Step [3144/3236], Loss: 2.3072, Per

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.